In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names: Sherry, Kelly, Ifan
### Team Strategy Chosen: SAFE

In [7]:
tickers_df = pd.read_csv('Tickers_Example.csv')
#When the CSV file is read, it takes the first ticker and makes it the name of the column, so we need to extract it
tickers_list = list(tickers_df.columns)
#Now add the rest of the tickers in after it:
tickers_list += tickers_df.iloc[:, 0].tolist()
tickers_num = len(tickers_list)

#Dates to pull closing data from, change as needed
start_date = '2020-01-01'
end_date = '2022-01-02'

# Dictionary of all the stocks
stocks_dict = {}
stocks_full_info = {}

def get_closing_prices(ticker, start, end):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end) #Daily
    return hist.Close

for i in range (tickers_num):
    stocks_dict[tickers_list[i]] = get_closing_prices(tickers_list[i], start_date, end_date)
    stocks_full_info[tickers_list[i]] = yf.Ticker(tickers_list[i])

# Create DataFrame
prices = pd.DataFrame(stocks_dict)

#Some tickers are delisted, so we drop those columns entirely
for i in range (tickers_num):
    if prices[tickers_list[i]].isnull().all():
        prices.drop(columns=[tickers_list[i]], inplace=True)

#Some of the stocks are delisted, so if we uncomment this now, it'll delete the entire DataFrame
prices = prices.dropna() 

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(prices)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,BK,BLK,BMY,C,CAT,CL,CMCSA,COF,COP,COST,CSCO,CVS,GM,GOOG,JPM,KMI,KO,LLY,LMT,MO,MON,MRK,MS,MSFT,NEE,NKE,ORCL,OXY,PEP,PFE,PG,PM,PYPL,QCOM,RY.TO,SBUX,SLB,SO,SPG,T,TD.TO,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-16 00:00:00,124.265907,102.991295,114.960403,257.676666,45.678761,154.593002,140.325348,255.210007,36.480053,259.920013,43.974258,698.315796,58.655914,68.990318,217.884521,72.816872,55.796638,124.143265,52.917690,323.250336,47.165886,72.076775,56.989883,104.625999,146.212952,14.289574,48.974056,186.289185,332.890930,43.404175,9.785,69.581032,78.276802,234.083542,73.130440,142.769943,64.960365,27.963650,128.622879,33.759583,123.469925,80.504601,249.309998,129.892365,108.809334,107.442192,27.394434,57.139629,107.247581,19.613207,76.842140,174.705948,169.330276,347.556396,205.282013,155.261246,50.686962
2021-03-17 00:00:00,123.464317,97.601990,114.376961,258.410095,46.006893,156.786499,141.676697,263.589996,36.780506,264.149994,44.231026,703.289490,58.921234,69.244339,224.783005,72.759285,55.277504,127.196350,53.151501,325.166565,46.928486,71.059044,59.913212,104.554001,147.717865,14.558847,48.993176,182.219879,336.621368,43.702663,9.680,70.024918,79.528610,233.423767,70.782661,142.937759,64.348160,28.827276,128.248566,33.721893,122.971985,81.417351,251.470001,130.154800,109.203346,106.593010,27.638504,56.626534,108.208496,19.717848,76.935684,172.951248,171.442856,345.184448,205.863831,153.191986,51.298893
2021-03-18 00:00:00,119.278252,96.421921,114.853439,260.971985,46.151665,151.399506,139.620300,256.059998,37.739994,262.269989,44.411716,698.953247,59.063370,69.470123,221.944702,73.018456,53.710514,123.969345,49.924812,319.032562,46.339748,71.510292,59.134987,101.810997,150.156158,14.145961,48.352547,179.487503,343.746613,44.396221,9.770,69.997742,79.604492,227.200180,69.007286,141.309189,64.853477,26.425003,127.202385,33.703056,123.240112,82.658684,238.410004,126.102028,109.794327,103.852425,26.603647,56.346668,106.908966,19.626287,77.655937,174.124283,166.672821,354.858337,208.831070,152.762863,52.974648
2021-03-19 00:00:00,118.743851,96.096710,116.934433,257.930939,45.196205,153.748001,137.789139,255.820007,37.342628,267.070007,44.078869,704.660889,59.413979,68.689255,216.758820,71.972176,53.316360,122.152939,49.195305,324.890045,46.510677,70.262131,59.683735,102.160004,147.765503,14.289574,48.582031,179.838806,340.505249,44.378670,9.750,70.215141,78.656143,226.835831,68.725937,135.703033,64.387039,27.894163,129.093201,33.476913,122.579369,82.556938,241.279999,127.336327,109.325294,102.617233,26.408390,55.693645,106.790001,19.462788,77.094704,181.821777,168.221405,358.318268,204.409317,151.895096,51.976723
2021-03-22 00:00:00,122.108559,98.401100,118.120804,259.319458,44.617138,155.543503,137.044876,251.229996,36.499439,272.339996,42.994724,705.897034,60.058350,67.701378,217.461166,72.730469,54.191185,120.983879,48.858608,330.401825,47.764130,70.540573,57.967651,101.929497,143.793701,14.163911,48.763695,181.351395,342.989014,45.335590,9.700,70.215141,77.688828,232.389801,69.550575,136.472900,64.445343,27.258850,132.260544,33.919758,124.657310,83.232201,244.384995,130.319992,108.799950,103.804184,26.994160,55.684315,105.472168,19.613207,76.832787,182.296814,172.159744,359.572815,200.016663,153.582947,51.063534
2021-03-23 00:00:00,121.267380,97.416153,117.167824,260.131012,43.410744,156.875000,133.255234,241.250000,35.762867,267.209991,42.195885,692.038330,59.338173,66.713539,209.985397,74.631050,53.970074,118.268944,48.362907,336.180328,47.488747,69.033173,56.032074,102.648003,142.355499,14.074154,49.136597,179.243546,339.105133,44.343544,9.660,69.091850,75.033447,233.955521,71.811035,135.337830,65.368484,26.395224,133.853821,33.316742,126.974670,81.446922,243.770004,128.794128,108.668625,102.530396,26.037405,56.561230,101.042801,19.619745,76.439934,183.149948,170.897980,361.180267,201.742691,152.438660,50.442184
2021-03-24 00:

In [13]:
stocks_full_info

{'AAPL': yfinance.Ticker object <AAPL>,
 'ABBV': yfinance.Ticker object <ABBV>,
 'ABT': yfinance.Ticker object <ABT>,
 'ACN': yfinance.Ticker object <ACN>,
 'AGN': yfinance.Ticker object <AGN>,
 'AIG': yfinance.Ticker object <AIG>,
 'AMZN': yfinance.Ticker object <AMZN>,
 'AXP': yfinance.Ticker object <AXP>,
 'BA': yfinance.Ticker object <BA>,
 'BAC': yfinance.Ticker object <BAC>,
 'BIIB': yfinance.Ticker object <BIIB>,
 'BK': yfinance.Ticker object <BK>,
 'BLK': yfinance.Ticker object <BLK>,
 'BMY': yfinance.Ticker object <BMY>,
 'C': yfinance.Ticker object <C>,
 'CAT': yfinance.Ticker object <CAT>,
 'CELG': yfinance.Ticker object <CELG>,
 'CL': yfinance.Ticker object <CL>,
 'CMCSA': yfinance.Ticker object <CMCSA>,
 'COF': yfinance.Ticker object <COF>,
 'COP': yfinance.Ticker object <COP>,
 'COST': yfinance.Ticker object <COST>,
 'CSCO': yfinance.Ticker object <CSCO>,
 'CVS': yfinance.Ticker object <CVS>,
 'GM': yfinance.Ticker object <GM>,
 'GOOG': yfinance.Ticker object <GOOG>,
 'JP

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.